## Imports

In [1]:
import pandas as pd
import json
from pyteomics import fasta
import random

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import runner

# Hacky False Positive Calculation
For the sake of making it easier, we are going to do the FP in a bit of a weird way. We have a dataset with 280 proteins we know to be the acutal source. We will run hypedsearch with both this set and a set of 300 prots that are NOT in this set. Then we will compare results. 
1. Load in the truth set
2. Create another set of proteins that are exclusive of the truth set
3. Run hypedsearch on both sets
4. Count the number of results that the non truth set had over the truth set

## Constants

In [2]:
specPath = '/Users/zacharymcgrath/Desktop/nod2 data/filteredSpec/'

truth_set = '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta'
not_truth_set = '/Users/zacharymcgrath/Desktop/nod2 data/not_truth_subset.fasta'
whole_db = '/Users/zacharymcgrath/Desktop/nod2 data/all data/NOD2_mouse_database.fasta'

outputDir = '/Users/zacharymcgrath/Desktop/Experiment output/FP/'
minPep = 3
maxPep = 30
tolerance = 20
relative_abundance_filter = 0.0
precursor_tolerance = 3
peak_filter = 25
verbose = True

truth_run_params = {
    'spectra_folder': specPath,
    'database_file': truth_set,
    'output_dir': outputDir + 'truth/',
    'min_peptide_len': minPep,
    'max_peptide_len': maxPep,
    'tolerance': tolerance,
    'precursor_tolerance': precursor_tolerance,
    'peak_filter': peak_filter, 
    'relative_abundance_filter': relative_abundance_filter,
    'verbose': verbose,
    'DEBUG': False
}

non_truth_run_params = {
    'spectra_folder': specPath,
    'database_file': not_truth_set,
    'output_dir': outputDir + 'not_truth/',
    'min_peptide_len': minPep,
    'max_peptide_len': maxPep,
    'tolerance': tolerance,
    'precursor_tolerance': precursor_tolerance,
    'peak_filter': peak_filter, 
    'relative_abundance_filter': relative_abundance_filter,
    'verbose': verbose, 
    'DEBUG': False
}

## 1. Load in truth set

In [3]:
get_name = lambda x: x.split('|')[-1].split(' ')[0]

ts = {get_name(entry.description): None for entry in fasta.read(truth_set)}

## 2. Create the other set of 280 proteins

In [4]:
num_keep = 280
all_prots = {get_name(entry.description): entry for entry in fasta.read(whole_db)}
all_prot_keys = list(all_prots.keys())

saving = {}
# get num_keep
for i in range(num_keep):
    print(f'\rchoosing prot {i+1}/{num_keep}', end='')
    
    while not False:
        
        # pick a protein 
        selected = all_prot_keys[random.randint(0, len(all_prot_keys) - 1)]
        
        # see if its in saving or ts
        if selected in ts or selected in saving:
            continue
            
        added = True
        saving[selected] = all_prots[selected]
        break
            
# write to file
fasta.write([v for _, v in saving.items()], open(not_truth_set, 'w'))

choosing prot 280/280

<_io.TextIOWrapper name='/Users/zacharymcgrath/Desktop/nod2 data/not_truth_subset.fasta' mode='w' encoding='UTF-8'>

## 3. Run hyped search on both

In [ ]:
print(truth_run_params)
runner.run(truth_run_params)

{'spectra_folder': '/Users/zacharymcgrath/Desktop/nod2 data/filteredSpec/', 'database_file': '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta', 'output_dir': '/Users/zacharymcgrath/Desktop/Experiment output/FP/truth/', 'min_peptide_len': 3, 'max_peptide_len': 30, 'tolerance': 20, 'precursor_tolerance': 3, 'peak_filter': 25, 'relative_abundance_filter': 0.0, 'verbose': True, 'DEBUG': False}
Loading database...
Done
Loading spectra...


In [ ]:
runner.run(non_truth_run_params)